In [ ]:
from transformers.agents import Tool
from sqlalchemy import create_engine, text, inspect
from transformers.agents import ReactCodeAgent, HfEngine

# Conecte ao BD e obtenha os detalhes da tabela
engine = create_engine("<DB CONNECTION STRING>")
inspector = inspect(engine)
columns_info = [(col["name"], col["type"]) 
                for col in inspector.get_columns("receipts")]

table_description = "Columns:\n" + "\n".join([f" - {name}: {col_type}" 
                              for name, col_type in columns_info])

# Cria uma ferramenta de execução SQL
class SQLExecutorTool(Tool):
    name = "sql_engine"
    description = f"""Allows you to perform SQL queries on the table.
    Returns a string representation of the result.
    The table is named 'receipts'. Description is as follows:
    {table_description}
    """
    
    inputs = {{
        "query": {{"type": "text",
                  "description": f"The query to perform. This should be correct SQL."}},
    }}
    output_type = "text"
    
    def forward(self, query: str) -> str:
        output = ""
        try:
            with engine.connect() as con:
                rows = con.execute(text(query))
                for row in rows:
                    output += "\n" + str(row)
        except Exception as e:
            output = f"An error occurred: {str(e)}"
        finally:
            con.close()  # Ensure the connection is closed
        return output

# Cria o agente ReAct
agent = ReactCodeAgent(
    tools=[SQLExecutorTool()],
    llm_engine=HfEngine("meta-llama/Meta-Llama-3-70B-Instruct"),
)

# Executa o agente
agent.run("Give me the name of the client who got the most expensive receipt?")
